In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [142]:
from langchain_teddynote import logging

logging.langsmith("structured")

LangSmith 추적을 시작합니다.
[프로젝트명]
structured


### 모듈 import

In [3]:
import pandas as pd
import os
from dotenv import load_dotenv
from langchain_teddynote import logging
from langchain.text_splitter import CharacterTextSplitter
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_anthropic import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda 
from langchain.memory import ConversationBufferMemory
from langchain.schema import Document
from datetime import datetime
from langchain_community.document_loaders import JSONLoader
import json
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.chat_models import ChatAnthropic
from langchain.schema.output_parser import StrOutputParser

### 1. 대분류 :의도파악하기

In [4]:
intent_chain = PromptTemplate.from_template("""주어진 아래의 질문을 보고 '주문','취소','결제','기타' 중 하나로 분류하세요.                                     
하나의 단어 이상 표현하지 마세요. 

주문 : 주문하고자 하는 의도가 있는 경우, 메뉴 추천을 받고 싶은 경우
취소 : 주문 취소를 의도한 경우, 주문 도중 다른 메뉴를 원하는 경우
결제 : 주문이 완료된 경우, 결제를 의도한 경우 
채팅 : 위의 네 가지 의도 외의 경우

<질문>
{question}
</질문>

분류:""") | ChatUpstage() | StrOutputParser()

In [168]:
intent_chain.invoke("햄버거가 먹고싶어")
# 주문

'주문.'

In [169]:
intent_chain.invoke("상하이 스파이시 버거 하나 줘")
# 주문

'주문.'

In [170]:
intent_chain.invoke("주문할게")
# 주문

'주문.'

In [171]:
intent_chain.invoke("뭐 먹을지 잘 모르겠어")
# 주문

'기타'

In [172]:
intent_chain.invoke("이거 어떻게 하는거야?")
# 기타

'기타'

In [173]:
intent_chain.invoke("방금 얘기한거 주문 안할게")
# 취소

'취소'

In [174]:
intent_chain.invoke("주문 끝났어")
# 결제

'주문.'

In [175]:
intent_chain.invoke("계산해줘")
# 결제

'결제'

In [176]:
intent_chain.invoke("매콤한거 먹을래")
# 주문

'기타'

In [178]:
intent_chain.invoke("처음부터 주문 다시할게")
# 취소

'취소.'

In [ ]:
intent_chain.invoke("")

### task별 prompt

주문 : 세트, 단품 분리하고 시작?  -> RAG 시스템 통으로 이용  
결제, 취소는 코딩으로 해결  
기타 : LLM 무한가동시켜서 의도분류 task중 하나 선택하도록 유도  

In [1]:
file_dir = "/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/menu_1014.json"

with open(file_dir, 'r', encoding='utf-8') as file:
    content = file.read()
    # JSON 객체들을 리스트로 변환
    json_objects = json.loads(content)

# 파싱된 JSON 객체들을 문서로 변환
docs = [
    Document(
        page_content=json.dumps(obj['page_content'], ensure_ascii=False),
    )
    for obj in json_objects
]


file_name = file_dir.split("/")[-1]
cache_dir = LocalFileStore(f"./.cache/embeddings/{file_name}")

# CharacterTextSplitter를 사용하여 문서 분할
text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=100, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)

embeddings = UpstageEmbeddings(
    model="solar-embedding-1-large"
)

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embeddings,
    document_embedding_cache=cache_dir,
    namespace="solar-embedding-1-large",
)
vectorstore = FAISS.from_documents(
    split_docs,
    cached_embedder,
)

NameError: name 'json' is not defined

In [ ]:
faiss = vectorstore.as_retriever(search_kwargs={"k": 4})

bm25 = BM25Retriever.from_documents(split_docs)
bm25.k = 4

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25, faiss],
    weights=[0.3, 0.7],
    search_type="mmr",
)

In [ ]:
from langchain.prompts import ChatPromptTemplate

order_template = ChatPromptTemplate.from_messages([
    ("system",
    """
    당신은 맥도날드 가게의 점원입니다. 
    나이가 많은 노인 고객의 주문을 도와주세요.
    나이와 관련된 어떤한 호칭도 하지 말고 '고객님'으로 부르세요.
    어린아이도 이해하기 쉬운 단어로 설명해주세요.
    상냥하고, 친절하고, 존댓말로 사용자의 질문에 답변을 해주세요. 
    간결하게 답변하며, 가능한 한 문장 이내로 대답해주세요.  
    
    반드시 제공된 정보만을 사용하여 질문에 대답하세요.
    질문과 가장 관련성이 높은 정보를 찾아서 답변하세요.
    충분한 정보를 바탕으로 정확히 답변할 수 있는 경우에만 답변하세요.

    답변이 확실하지 않을 경우, "죄송합니다. 해당 질문에 대한 답을 찾을 수 없습니다."라고 답하세요.
    
    주문의 절차는 다음과 같습니다 : 
    1. 고객의 주문을 받습니다.
    2. 버거 메뉴를 선택할 때 항상 세트 메뉴를 원하는지 확인합니다. 
       세트 메뉴는 버거, 사이드, 음료가 포함됩니다. 
       기본 사이드는 후렌치 후라이 1개, 음료는 콜라 1개입니다. 사이드와 음료는 변경 가능합니다.
       다른 메뉴로 변경 시 추가 금액을 안내해 드립니다.
    4. 주문 완료 시 주문을 검토합니다.
    5. 주문 결과를 json 형식으로 출력합니다.

    주문 결과 예시: 
    {{
        "주문 메뉴" : [
            {{
                "메뉴 이름" : "베이컨 토마토 디럭스",
                "추가 옵션" : "후렌치 후라이 1개, 코카콜라 1개",
                "단품 금액" : 6600,
                "추가 금액" : 5600,
                "주문 금액" : 12200
            }}
        ]
    }}
    Context: {context}
    """),
    ("human", "{question}"),
])

In [ ]:
llm = ChatAnthropic(model_name="claude-3-5-sonnet-20240620")

order_chain = {
    "context": ensemble_retriever,
    "question": RunnablePassthrough()
} | order_template | llm

In [ ]:
order_chain.invoke("치킨버거 주문할게")

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': '"claude-3-5-sonnet-20240620" is not supported on this API. Please use the Messages API instead.'}}